# MusicoSet Metadata Cleaning

Clean tab-separated musicoset files - convert `-` and `[]` to NA, trim whitespace

In [15]:
library(tidyverse)
library(readr)

In [16]:
# load artists
artists_in <- '../../data/musicoset_metadata/artists.csv'
songs_in <- '../../data/musicoset_metadata/songs.csv'
artists_out <- '../../data/cleaned/musicoset_artists_cleaned.csv'
songs_out <- '../../data/cleaned/musicoset_songs_cleaned.csv'

artists <- read_delim(artists_in, delim="\t", na=c("", "NA", "-"))
sprintf("%d artists loaded", nrow(artists))

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 11518 Columns: 8
-- Column specification --------------------------------------------------------
Delimiter: "\t"
chr (6): artist_id, name, artist_type, main_genre, genres, image_url
dbl (2): followers, popularity

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "11518 artists loaded"

In [17]:
glimpse(artists)

Rows: 11,518
Columns: 8
$ artist_id   <chr> "66CXWjxzNUsdJxJ2JdwvnR", "26VFTg2z8YR0cCuwLzESi2", "0Y5tJ~
$ name        <chr> "Ariana Grande", "Halsey", "Travis Scott", "Post Malone", ~
$ followers   <dbl> 34554242, 7368242, 6313709, 16737002, 483032, 15566666, 37~
$ popularity  <dbl> 96, 90, 94, 96, 89, 91, 85, 88, 81, 89, 86, 90, 88, 62, 55~
$ artist_type <chr> "singer", "singer", "rapper", "rapper", "singer", "DJ", "b~
$ main_genre  <chr> "dance pop", "dance pop", "pop", "dfw rap", "trap music", ~
$ genres      <chr> "['dance pop', 'pop', 'post-teen pop']", "['dance pop', 'e~
$ image_url   <chr> "https://i.scdn.co/image/b1dfbe843b0b9f54ab2e588f33e7637d2~


In [18]:
# load songs
songs <- read_delim(songs_in, delim="\t", na=c("", "NA", "-"))
sprintf("%d songs loaded", nrow(songs))

Rows: 20405 Columns: 7
-- Column specification --------------------------------------------------------
Delimiter: "\t"
chr (5): song_id, song_name, billboard, artists, song_type
dbl (1): popularity
lgl (1): explicit

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "20405 songs loaded"

In [19]:
glimpse(songs)

Rows: 20,405
Columns: 7
$ song_id    <chr> "3e9HZxeyfWwjeyPAMmWSSQ", "5p7ujcrUXASCNwRaWNHR1C", "2xLMif~
$ song_name  <chr> "thank u, next", "Without Me", "SICKO MODE", "Sunflower - S~
$ billboard  <chr> "('Thank U, Next', 'Ariana Grande')", "('Without Me', 'Hals~
$ artists    <chr> "{'66CXWjxzNUsdJxJ2JdwvnR': 'Ariana Grande'}", "{'26VFTg2z8~
$ popularity <dbl> 86, 87, 85, 92, 86, 63, 52, 53, 3, 51, 81, 48, 78, 77, 86, ~
$ explicit   <lgl> TRUE, TRUE, TRUE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,~
$ song_type  <chr> "Solo", "Solo", "Solo", "Collaboration", "Solo", "Solo", "S~


In [29]:
artists2 %>% 
  summarise(across(everything(), ~sum(is.na(.)))) %>% 
  pivot_longer(everything()) %>%
  arrange(desc(value))

name,value
<chr>,<int>
artist_type,4466
main_genre,3148
genres,3148
image_url,504
followers,2
artist_id,0
name,0
popularity,0


In [30]:
# verify no standalone dashes or empty brackets
check <- artists2 %>% filter(if_any(where(is.character), ~ . == "-" | . == "[]"))
if(nrow(check) == 0) print("✓ No dashes or []")

[1] "<U+2713> No dashes or []"


In [20]:
# clean artists - trim and fix dashes/empty lists
fix_dash <- function(x) {
  if(is.character(x)) {
    x <- ifelse(str_detect(x, "^-$"), NA_character_, x)
    x <- ifelse(str_detect(x, "^\\[\\]$"), NA_character_, x)
    x
  } else x
}

artists2 <- artists %>% 
  mutate(across(where(is.character), str_trim)) %>%
  mutate(across(where(is.character), fix_dash))

sprintf("Cleaned %d artists", nrow(artists2))

[1] "Cleaned 11518 artists"

In [21]:
# missing data (after cleaning - includes both - and [])
artists2 %>% 
  summarise(across(everything(), ~sum(is.na(.)))) %>% 
  pivot_longer(everything()) %>%
  arrange(desc(value))

name,value
<chr>,<int>
artist_type,4466
main_genre,3148
genres,3148
image_url,504
followers,2
artist_id,0
name,0
popularity,0


In [22]:
# verify no standalone dashes or empty brackets
check <- artists2 %>% filter(if_any(where(is.character), ~ . == "-" | . == "[]"))
if(nrow(check) == 0) print("✓ No dashes or []")

[1] "<U+2713> No dashes or []"


In [23]:
# clean songs
songs2 <- songs %>% 
  mutate(across(where(is.character), str_trim)) %>%
  mutate(across(where(is.character), fix_dash))

sprintf("Cleaned %d songs", nrow(songs2))

[1] "Cleaned 20405 songs"

In [24]:
# song type breakdown
songs2 %>% count(song_type, sort=T)

song_type,n
<chr>,<int>
Solo,18978
Collaboration,1427


In [25]:
# export
dir.create("../../data/cleaned", showWarnings=F, recursive=T)

write_csv(artists2, artists_out, na="", quote="needed")
write_csv(songs2, songs_out, na="", quote="needed")

sprintf("Wrote artists: %s", artists_out)
sprintf("Wrote songs: %s", songs_out)

[1] "Wrote artists: ../../data/cleaned/musicoset_artists_cleaned.csv"

[1] "Wrote songs: ../../data/cleaned/musicoset_songs_cleaned.csv"

In [26]:
# verify files
a_lines <- length(readLines(artists_out))
s_lines <- length(readLines(songs_out))

sprintf("Artists: %d lines (expected %d)", a_lines, nrow(artists2)+1)
sprintf("Songs: %d lines (expected %d)", s_lines, nrow(songs2)+1)

[1] "Artists: 11519 lines (expected 11519)"

[1] "Songs: 20406 lines (expected 20406)"

In [27]:
# preview artists output
artists2 %>% 
  select(name, followers, popularity, artist_type, main_genre) %>%
  head(10)

name,followers,popularity,artist_type,main_genre
<chr>,<dbl>,<dbl>,<chr>,<chr>
Ariana Grande,34554242,96,singer,dance pop
Halsey,7368242,90,singer,dance pop
Travis Scott,6313709,94,rapper,pop
Post Malone,16737002,96,rapper,dfw rap
Swae Lee,483032,89,singer,trap music
Marshmello,15566666,91,DJ,brostep
Bastille,3776115,85,band,metropopolis
Panic! At The Disco,7749228,88,band,baroque pop
Mariah Carey,4682308,81,singer,dance pop


In [28]:
# preview songs output  
songs2 %>%
  select(song_name, billboard, popularity, explicit, song_type) %>%
  head(10)

song_name,billboard,popularity,explicit,song_type
<chr>,<chr>,<dbl>,<lgl>,<chr>
"thank u, next","('Thank U, Next', 'Ariana Grande')",86,TRUE,Solo
Without Me,"('Without Me', 'Halsey')",87,TRUE,Solo
SICKO MODE,"('Sicko Mode', 'Travis Scott')",85,TRUE,Solo
Sunflower - Spider-Man: Into the Spider-Verse,"('Sunflower (Spider-Man: Into The Spider-Verse)', 'Post Malone & Swae Lee')",92,FALSE,Collaboration
High Hopes,"('High Hopes', 'Panic! At The Disco')",86,FALSE,Solo
All I Want for Christmas Is You,"('All I Want For Christmas Is You', 'Mariah Carey')",63,FALSE,Solo
It's the Most Wonderful Time of the Year,"(""It's The Most Wonderful Time Of The Year"", 'Andy Williams')",52,FALSE,Solo
Rockin' Around The Christmas Tree,"(""Rockin' Around The Christmas Tree"", 'Brenda Lee')",53,FALSE,Solo
A Holly Jolly Christmas,"('A Holly Jolly Christmas', 'Burl Ives')",3,FALSE,Solo
